In [3]:
import numpy as np
import os

import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import classification_report, confusion_matrix

from pymongo import MongoClient

import datetime

import sys
sys.path.append('../..')

import utils.dbUtils
import utils.gensimUtils

client = MongoClient('localhost', 27017)
db = client.TFE
collection = db.results

In [4]:
print('Creating corpus')
corpus = utils.dbUtils.TokenizedIterator('news_cleaned', filters = {'type' : {'$in' : ['fake', 'reliable']}}, limit = 2000)
print('Creating labels')
y = np.array([x for x in corpus.iterTags()])

Creating corpus
Creating labels


In [5]:
train_accuracy = []
test_accuracy = []
kf = KFold(n_splits=3, shuffle = True)
i = 1
for i, (train_index, test_index) in enumerate(kf.split(y)):
    print('Train and test set {}'.format(i))
    model = LinearSVC()
    vectorizer = TfidfVectorizer()
    print('\t Fiting tf-idf')
    X_train = vectorizer.fit_transform([' '.join(corpus[i]) for i in train_index])
    X_test = vectorizer.transform([' '.join(corpus[i]) for i in test_index])
    y_train = y[train_index]
    y_test = y[test_index]
    print('\t fiting model')
    model.fit(X_train, y_train)
    print('\t Testing model')
    train_accuracy.append(model.score(X_train, y_train))
    test_accuracy.append(model.score(X_test, y_test))
    #print("Training accuracy : {}".format(model.score(X_train, y_train)))
    #print("Test accuracy : {}".format(model.score(X_test, y_test)))
    #print("Classification report for test set")
    #print(classification_report(y_test, model.predict(X_test)))
    crp = classification_report(y_test, model.predict(X_test), labels=['fake', 'reliable'], output_dict = True)
    print(crp)

Train and test set 0
	 Fiting tf-idf
	 fiting model
	 Testing model
{'fake': {'precision': 0.9517684887459807, 'recall': 0.9966329966329966, 'f1-score': 0.9736842105263157, 'support': 594}, 'reliable': {'precision': 0.9555555555555556, 'recall': 0.589041095890411, 'f1-score': 0.728813559322034, 'support': 73}, 'micro avg': {'precision': 0.952023988005997, 'recall': 0.952023988005997, 'f1-score': 0.952023988005997, 'support': 667}, 'macro avg': {'precision': 0.9536620221507681, 'recall': 0.7928370462617038, 'f1-score': 0.8512488849241748, 'support': 667}, 'weighted avg': {'precision': 0.9521829653233406, 'recall': 0.952023988005997, 'f1-score': 0.9468842741876163, 'support': 667}}
Train and test set 1
	 Fiting tf-idf
	 fiting model
	 Testing model
{'fake': {'precision': 0.9712, 'recall': 0.9967159277504105, 'f1-score': 0.9837925445705025, 'support': 609}, 'reliable': {'precision': 0.9523809523809523, 'recall': 0.6896551724137931, 'f1-score': 0.7999999999999999, 'support': 58}, 'micro av